In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

%matplotlib inline

In [146]:
logs = pd.read_csv("all-game-logs.csv")
shots = pd.read_csv("shotcharts-2018-19.csv")
pbp = pd.read_csv("pbp-2018-19.csv")

In [184]:
pbp.tail(100)

,Unnamed: 0,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,next_Home_desc,next_Visitor_desc,two_next_play_event_type,two_next_Home_desc,two_next_Visitor_desc,creates_OREB,OREB_outcome,two_next_shot_value
96743,359,21800201,533,6,1,4,12:24 AM,10:25,Jerebko P.FOUL (P3.T1) (D.Collins),NaN,NaN,NaN,NaN,4.0,201973,Jonas Jerebko,1.610613e+09,Golden State,Warriors,GSW,5,1627816,Alex Poythress,1.610613e+09,Atlanta,Hawks,ATL,1,0,NaN,NaN,NaN,NaN,NaN,NaN,SUB: Len FOR Plumlee,1.0,NaN,Poythress 3' Cutting Dunk Shot (2 PTS) (Bembry...,0,0,0
96744,360,21800201,535,8,0,4,12:24 AM,10:25,NaN,NaN,SUB: Len FOR Plumlee,NaN,NaN,5.0,203101,Miles Plumlee,1.610613e+09,Atlanta,Hawks,ATL,5,203458,Alex Len,1.610613e+09,Atlanta,Hawks,ATL,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Poythress 3' Cutting Dunk Shot (2 PTS) (Bembry...,2.0,MISS Looney 6' Jump Shot,Bembry BLOCK (1 BLK),0,0,0
96745,361,21800201,537,1,108,4,12:25 AM,10:14,NaN,NaN,Poythress 3' Cutting Dunk Shot (2 PTS) (Bembry...,78 - 86,8,5.0,1627816,Alex Poythress,1.610613e+09,Atlanta,Hawks,ATL,5,1627761,DeAndre' Bembry,1.610613e+09,Atlanta,Hawks,ATL,0,0,NaN,NaN,NaN,NaN,NaN,MISS Looney 6' Jump Shot,Bembry BLOCK (1 BLK),4.0,Looney REBOUND (Off:2 Def:1),NaN,0,0,0
96746,362,21800201,539,2,1,4,12:25 AM,9:51,MISS Looney 6' Jump Shot,NaN,Bembry BLOCK (1 BLK),NaN,NaN,4.0,1626172,Kevon Looney,1.610613e+09,Golden State,Warriors,GSW,0,0,NaN,NaN,NaN,NaN,NaN,5,1627761,DeAndre' Bembry,1.610613e+09,Atlanta,Hawks,ATL,Looney REBOUND (Off:2 Def:1),NaN,1.0,Looney 1' Dunk (3 PTS),NaN,1,1,2
96747,363,21800201,541,4,0,4,12:25 AM,9:50,Looney REBOUND (Off:2 Def:1),NaN,NaN,NaN,NaN,4.0,1626172,Kevon Looney,1.610613e+09,Golden State,Warriors,GSW,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Looney 1' Dunk (3 PTS),NaN,6.0,NaN,Len S.FOUL (P4.T1) (M.Callahan),0,0,0
96748,364,21800201,542,1,7,4,12:25 AM,9:49,Looney 1' Dunk (3 PTS),NaN,NaN,78 - 88,10,4.0,1626172,Kevon Looney,1.610613e+09,Golden State,Warriors,GSW,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Len S.FOUL (P4.T1) (M.Callahan),3.0,Looney Free Throw 1 of 1 (4 PTS),NaN,0,0,0
96749,365,21800201,543,6,2,4,12:25 AM,9:49,NaN,NaN,Len S.FOUL (P4.T1) (M.Callahan),NaN,NaN,5.0,203458,Alex Len,1.610613e+09,Atlanta,Hawks,ATL,4,1626172,Kevon Looney,1.610613e+09,Golden State,Warriors,GSW,1,0,NaN,NaN,NaN,NaN,NaN,Looney Free Throw 1 of 1 (4 PTS),NaN,2.0,NaN,MISS Young 8' Floating Jump Shot,0,0,0
96750,366,21800201,545,3,10,4,12:26 AM,9:49,Looney Free Throw 1 of 1 (4 PTS),NaN,NaN,78 - 89,11,4.0,1626172,Kevon Looney,1.610613e+09,Golden State,Warriors,GSW,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,MISS Young 8' Floating Jump Shot,4.0,NaN,Poythress REBOUND (Off:2 Def:1),0,0,0
96751,367,21800201,546,2,78,4,12:26 AM,9:40,NaN,NaN,MISS Young 8' Floating Jump Shot,NaN,NaN,5.0,1629027,Trae Young,1.610613e+09,Atlanta,Hawks,ATL,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Poythress REBOUND (Off:2 Def:1),1.0,NaN,Poythress 1' Tip Layup Shot (4 PTS),1,1,2
96752,368,21800201,547,4,0,4,12:26 AM,9:39,NaN,NaN,Poythress REBOUND (Off:2 Def:1),NaN,NaN,5.0,1627816,Alex Poythress,1.610613e+09,Atlanta,Hawks,ATL,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Poythress 1' Tip Layup Shot (4 PTS),2.0,MISS Livingston 11' Jump Shot,NaN,0,0,0


In [148]:
shots.head()

,Unnamed: 0,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,EVENT_TYPE,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,0,Shot Chart Detail,21800001,114,201143,Al Horford,1610612738,Boston Celtics,1,3,47,Made Shot,Driving Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,2,24,-1,1,1,20181016,BOS,PHI
1,1,Shot Chart Detail,21800001,271,201143,Al Horford,1610612738,Boston Celtics,2,5,7,Made Shot,Alley Oop Layup shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,-2,11,1,1,20181016,BOS,PHI
2,2,Shot Chart Detail,21800001,308,201143,Al Horford,1610612738,Boston Celtics,2,2,24,Made Shot,Driving Finger Roll Layup Shot,2PT Field Goal,Restricted Area,Center(C),Less Than 8 ft.,1,-9,7,1,1,20181016,BOS,PHI
3,3,Shot Chart Detail,21800001,353,201143,Al Horford,1610612738,Boston Celtics,3,11,5,Made Shot,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),8-16 ft.,12,-114,53,1,1,20181016,BOS,PHI
4,4,Shot Chart Detail,21800001,580,201143,Al Horford,1610612738,Boston Celtics,4,7,24,Missed Shot,Fadeaway Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,40,57,1,0,20181016,BOS,PHI


### Filter pbp data for OREB
1. Start by shifting `HOMEDESCRIPTION` and `VISITORDESCRIPTION` to get the next play and create two new columns, `next_Home_desc` and `next_Visitor_desc`
2. Create a new column `creates_OREB` and filter for current description contains `MISS` and next description contains `REBOUND`. 1 if true and 0 if false so that can easily sum to get counts later.

In [149]:
pbp['next_Home_desc'] = pbp['HOMEDESCRIPTION'].shift(-1)
pbp['next_Visitor_desc'] = pbp['VISITORDESCRIPTION'].shift(-1)

pbp['two_next_play_event_type'] = pbp['EVENTMSGTYPE'].shift(-2)
pbp['two_next_Home_desc'] = pbp['HOMEDESCRIPTION'].shift(-2)
pbp['two_next_Visitor_desc'] = pbp['VISITORDESCRIPTION'].shift(-2)

In [150]:
pbp['creates_OREB'] = np.where(((pbp.next_Home_desc.str.contains('REBOUND'))&(pbp.HOMEDESCRIPTION.str.contains('MISS'))) |
                                ((pbp.next_Visitor_desc.str.contains('REBOUND'))&(pbp.VISITORDESCRIPTION.str.contains('MISS')))
                                 ,
                               1,
                                0)

pbp['OREB_outcome'] = np.where((pbp.creates_OREB == 1)&(pbp.two_next_play_event_type == 1),
                               1,
                                0)
pbp['two_next_shot_value'] = np.where((pbp.OREB_outcome == 1),
                                       np.where(((pbp.two_next_Home_desc.str.contains('3PT')) & (pd.notnull(pbp.HOMEDESCRIPTION))) |
                                                ((pbp.two_next_Visitor_desc.str.contains('3PT'))& (pd.notnull(pbp.VISITORDESCRIPTION))),
                                                  3,
                                                  2),
                                      0)
            
                               

In [151]:
pbp

,Unnamed: 0,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,next_Home_desc,next_Visitor_desc,two_next_play_event_type,two_next_Home_desc,two_next_Visitor_desc,creates_OREB,OREB_outcome,two_next_shot_value
0,0,21800001,4,10,0,1,8:03 PM,12:00,Jump Ball Horford vs. Embiid: Tip to Simmons,NaN,NaN,NaN,NaN,4.0,201143,Al Horford,1.610613e+09,Boston,Celtics,BOS,5,203954,Joel Embiid,1.610613e+09,Philadelphia,76ers,PHI,5,1627732,Ben Simmons,1.610613e+09,Philadelphia,76ers,PHI,NaN,MISS Covington 27' 3PT Jump Shot,4.0,CELTICS Rebound,NaN,0,0,0
1,1,21800001,7,2,1,1,8:03 PM,11:40,NaN,NaN,MISS Covington 27' 3PT Jump Shot,NaN,NaN,5.0,203496,Robert Covington,1.610613e+09,Philadelphia,76ers,PHI,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,CELTICS Rebound,NaN,2.0,MISS Tatum 25' 3PT Jump Shot,NaN,0,0,0
2,2,21800001,8,4,0,1,8:04 PM,11:40,CELTICS Rebound,NaN,NaN,NaN,NaN,2.0,1610612738,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,MISS Tatum 25' 3PT Jump Shot,NaN,4.0,NaN,Saric REBOUND (Off:0 Def:1),0,0,0
3,3,21800001,10,2,1,1,8:04 PM,11:15,MISS Tatum 25' 3PT Jump Shot,NaN,NaN,NaN,NaN,4.0,1628369,Jayson Tatum,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Saric REBOUND (Off:0 Def:1),5.0,Hayward STEAL (1 STL),Simmons Bad Pass Turnover (P1.T1),0,0,0
4,4,21800001,11,4,0,1,8:04 PM,11:13,NaN,NaN,Saric REBOUND (Off:0 Def:1),NaN,NaN,5.0,203967,Dario Saric,1.610613e+09,Philadelphia,76ers,PHI,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,Hayward STEAL (1 STL),Simmons Bad Pass Turnover (P1.T1),2.0,MISS Brown 2' Running Layup,NaN,0,0,0
5,5,21800001,12,5,1,1,8:04 PM,11:08,Hayward STEAL (1 STL),NaN,Simmons Bad Pass Turnover (P1.T1),NaN,NaN,5.0,1627732,Ben Simmons,1.610613e+09,Philadelphia,76ers,PHI,4,202330,Gordon Hayward,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,MISS Brown 2' Running Layup,NaN,4.0,NaN,Simmons REBOUND (Off:0 Def:1),0,0,0
6,6,21800001,14,2,41,1,8:05 PM,11:03,MISS Brown 2' Running Layup,NaN,NaN,NaN,NaN,4.0,1627759,Jaylen Brown,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Simmons REBOUND (Off:0 Def:1),1.0,NaN,Embiid 1' Running Layup (2 PTS) (Simmons 1 AST),0,0,0
7,7,21800001,16,4,0,1,8:05 PM,10:58,NaN,NaN,Simmons REBOUND (Off:0 Def:1),NaN,NaN,5.0,1627732,Ben Simmons,1.610613e+09,Philadelphia,76ers,PHI,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Embiid 1' Running Layup (2 PTS) (Simmons 1 AST),2.0,MISS Tatum 5' Driving Layup,NaN,0,0,0
8,8,21800001,17,1,41,1,8:05 PM,10:55,NaN,NaN,Embiid 1' Running Layup (2 PTS) (Simmons 1 AST),2 - 0,-2,5.0,203954,Joel Embiid,1.610613e+09,Philadelphia,76ers,PHI,5,1627732,Ben Simmons,1.610613e+09,Philadelphia,76ers,PHI,0,0,NaN,NaN,NaN,NaN,NaN,MISS Tatum 5' Driving Layup,NaN,4.0,NaN,Simmons REBOUND (Off:0 Def:2),0,0,0
9,9,21800001,19,2,6,1,8:05 PM,10:36,MISS Tatum 5' Driving Layup,NaN,NaN,NaN,NaN,4.0,1628369,Jayson Tatum,1.610613e+09,Boston,Celtics,BOS,0,0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,Simmons REBOUND (Off:0 Def:2),2.0,NaN,MISS Saric 26' 3PT Pullup Jump Shot,0,0,0


### Merge shotcharts and pbp data
Merge shots with pbp on `GAME_ID` and `EVENTNUM`

In [152]:
shotsPBP = shots.merge(pbp, left_on=["GAME_ID","GAME_EVENT_ID"], right_on=["GAME_ID","EVENTNUM"])


In [153]:
shotsPBP['combined_zone']=shotsPBP['SHOT_ZONE_BASIC']+'_'+shotsPBP['SHOT_ZONE_AREA']+'_'+shotsPBP['SHOT_ZONE_RANGE']+'_'+shotsPBP['SHOT_TYPE']
shotsPBP = shotsPBP[shotsPBP.SHOT_ZONE_RANGE != "Back Court Shot"]
uniquezones = shotsPBP.combined_zone.unique()
uniquezones.sort()
print(uniquezones)
print(uniquezones.shape)

['Above the Break 3_Center(C)_24+ ft._2PT Field Goal'
 'Above the Break 3_Center(C)_24+ ft._3PT Field Goal'
 'Above the Break 3_Left Side Center(LC)_24+ ft._2PT Field Goal'
 'Above the Break 3_Left Side Center(LC)_24+ ft._3PT Field Goal'
 'Above the Break 3_Right Side Center(RC)_24+ ft._2PT Field Goal'
 'Above the Break 3_Right Side Center(RC)_24+ ft._3PT Field Goal'
 'In The Paint (Non-RA)_Center(C)_8-16 ft._2PT Field Goal'
 'In The Paint (Non-RA)_Center(C)_Less Than 8 ft._2PT Field Goal'
 'In The Paint (Non-RA)_Left Side(L)_8-16 ft._2PT Field Goal'
 'In The Paint (Non-RA)_Right Side(R)_8-16 ft._2PT Field Goal'
 'Left Corner 3_Left Side(L)_24+ ft._3PT Field Goal'
 'Mid-Range_Center(C)_16-24 ft._2PT Field Goal'
 'Mid-Range_Center(C)_8-16 ft._2PT Field Goal'
 'Mid-Range_Left Side Center(LC)_16-24 ft._2PT Field Goal'
 'Mid-Range_Left Side Center(LC)_16-24 ft._3PT Field Goal'
 'Mid-Range_Left Side(L)_16-24 ft._2PT Field Goal'
 'Mid-Range_Left Side(L)_16-24 ft._3PT Field Goal'
 'Mid-Range_

In [166]:
# shotsPBP['combined_zone'] = np.where((shotsPBP.combined_zone == 'Above the Break 3_Center(C)_24+ ft._2PT Field Goal'),
#                                'Mid-Range_Left Side(L)_16-24 ft._2PT Field Goal',
#                                 shotsPBP['combined_zone'])

shotsPBP.loc[shotsPBP.combined_zone == 'Above the Break 3_Center(C)_24+ ft._2PT Field Goal', "combined_zone"] = 'Mid-Range_Left Side(L)_16-24 ft._2PT Field Goal'
shotsPBP.loc[shotsPBP.combined_zone == 'Above the Break 3_Left Side Center(LC)_24+ ft._2PT Field Goal', "combined_zone"] = 'Mid-Range_Left Side Center(LC)_16-24 ft._2PT Field Goal'

shotsPBP.loc[shotsPBP.combined_zone == 'Above the Break 3_Right Side Center(RC)_24+ ft._2PT Field Goal', "combined_zone"] = 'Mid-Range_Right Side Center(RC)_16-24 ft._2PT Field Goal'
shotsPBP.loc[shotsPBP.combined_zone == 'Mid-Range_Left Side Center(LC)_16-24 ft._3PT Field Goal', "combined_zone"] = 'Above the Break 3_Left Side Center(LC)_24+ ft._3PT Field Goal'
shotsPBP.loc[shotsPBP.combined_zone == 'Mid-Range_Left Side(L)_16-24 ft._3PT Field Goal', "combined_zone"] = 'Left Corner 3_Left Side(L)_24+ ft._3PT Field Goal'
shotsPBP.loc[shotsPBP.combined_zone == 'Mid-Range_Right Side Center(RC)_16-24 ft._3PT Field Goal', "combined_zone"] = 'Above the Break 3_Right Side Center(RC)_24+ ft._3PT Field Goal'
shotsPBP.loc[shotsPBP.combined_zone == 'Mid-Range_Right Side(R)_16-24 ft._3PT Field Goal', "combined_zone"] = 'Right Corner 3_Right Side(R)_24+ ft._3PT Field Goal'



In [167]:
uniquezones = shotsPBP.combined_zone.unique()
uniquezones.sort()
print(uniquezones)
print(uniquezones.shape)

['Above the Break 3_Center(C)_24+ ft._3PT Field Goal'
 'Above the Break 3_Left Side Center(LC)_24+ ft._3PT Field Goal'
 'Above the Break 3_Right Side Center(RC)_24+ ft._3PT Field Goal'
 'In The Paint (Non-RA)_Center(C)_8-16 ft._2PT Field Goal'
 'In The Paint (Non-RA)_Center(C)_Less Than 8 ft._2PT Field Goal'
 'In The Paint (Non-RA)_Left Side(L)_8-16 ft._2PT Field Goal'
 'In The Paint (Non-RA)_Right Side(R)_8-16 ft._2PT Field Goal'
 'Left Corner 3_Left Side(L)_24+ ft._3PT Field Goal'
 'Mid-Range_Center(C)_16-24 ft._2PT Field Goal'
 'Mid-Range_Center(C)_8-16 ft._2PT Field Goal'
 'Mid-Range_Left Side Center(LC)_16-24 ft._2PT Field Goal'
 'Mid-Range_Left Side(L)_16-24 ft._2PT Field Goal'
 'Mid-Range_Left Side(L)_8-16 ft._2PT Field Goal'
 'Mid-Range_Right Side Center(RC)_16-24 ft._2PT Field Goal'
 'Mid-Range_Right Side(R)_16-24 ft._2PT Field Goal'
 'Mid-Range_Right Side(R)_8-16 ft._2PT Field Goal'
 'Restricted Area_Center(C)_Less Than 8 ft._2PT Field Goal'
 'Right Corner 3_Right Side(R)_24+

### Calculate FG% for each zone
1. Sum `SHOT_ATTEMPTED_FLAG` for each zone to get FGA
2. Sum `SHOT_MADE_FLAG` for each zone to get FGM
3. Calculate FGM/FGA for each zone
1. Sum `creates_OREB` for each zone
2. Sum misses for each zone (FGA - FGM)
3. Calculate OREB%
4. Calculate `pts_per_OREB`
5. Calculate `xpps = ` $FG\% \times shot\_value + OREB\% \times pts\_per\_OREB$

In [170]:
league_by_zone = shotsPBP.groupby(by="combined_zone", as_index=False).agg({"SHOT_TYPE":"max",
                                                                           "SHOT_MADE_FLAG":"sum",
                                                                            "SHOT_ATTEMPTED_FLAG": "sum",
                                                                           "creates_OREB": "sum",
                                                                                    "two_next_shot_value":"sum"})

league_by_zone.rename(columns={"SHOT_ATTEMPTED_FLAG": "FGA",
                               "creates_OREB": "OREB",
                        "SHOT_MADE_FLAG":"FGM",
                            "two_next_shot_value":"2nd_Chance_Pts"}, inplace=True)
league_by_zone.fillna(0, inplace=True)
league_by_zone["FG%"] = league_by_zone.FGM/league_by_zone.FGA
league_by_zone["MISSES"] = league_by_zone.FGA - league_by_zone.FGM
league_by_zone["OREB%"] = league_by_zone.OREB/league_by_zone.MISSES
league_by_zone["shot_value"] = np.where(league_by_zone.SHOT_TYPE.str.contains('2PT') ,2,3)
league_by_zone["pts_per_OREB"] = league_by_zone["2nd_Chance_Pts"]/league_by_zone["OREB"]
league_by_zone.fillna(0, inplace=True)
league_by_zone['xpps'] = league_by_zone["FG%"] * league_by_zone.shot_value + league_by_zone["OREB%"]*league_by_zone.pts_per_OREB

league_by_zone.sort_values("xpps", ascending=False)

,combined_zone,SHOT_TYPE,FGM,FGA,OREB,2nd_Chance_Pts,FG%,MISSES,OREB%,shot_value,pts_per_OREB,xpps
16,Restricted Area_Center(C)_Less Than 8 ft._2PT ...,2PT Field Goal,7315,11736,1310,1294,0.623296,4421,0.296313,2,0.987786,1.539286
7,Left Corner 3_Left Side(L)_24+ ft._3PT Field Goal,3PT Field Goal,532,1378,160,123,0.386067,846,0.189125,3,0.768750,1.303590
17,Right Corner 3_Right Side(R)_24+ ft._3PT Field...,3PT Field Goal,494,1328,163,156,0.371988,834,0.195444,3,0.957055,1.303014
1,Above the Break 3_Left Side Center(LC)_24+ ft....,3PT Field Goal,1286,3603,434,377,0.356925,2317,0.187311,3,0.868664,1.233485
2,Above the Break 3_Right Side Center(RC)_24+ ft...,3PT Field Goal,1210,3453,407,299,0.350420,2243,0.181453,3,0.734644,1.184563
0,Above the Break 3_Center(C)_24+ ft._3PT Field ...,3PT Field Goal,868,2611,252,215,0.332440,1743,0.144578,3,0.853175,1.120670
5,In The Paint (Non-RA)_Left Side(L)_8-16 ft._2P...,2PT Field Goal,126,320,40,51,0.393750,194,0.206186,2,1.275000,1.050387
4,In The Paint (Non-RA)_Center(C)_Less Than 8 ft...,2PT Field Goal,1160,3009,483,451,0.385510,1849,0.261222,2,0.933747,1.014936
3,In The Paint (Non-RA)_Center(C)_8-16 ft._2PT F...,2PT Field Goal,744,1844,243,214,0.403471,1100,0.220909,2,0.880658,1.001487
15,Mid-Range_Right Side(R)_8-16 ft._2PT Field Goal,2PT Field Goal,354,859,97,89,0.412107,505,0.192079,2,0.917526,1.000452


In [179]:
team_by_zone = shotsPBP.groupby(by=["TEAM_ID","combined_zone"], as_index=False).agg({"SHOT_TYPE":"max",
                                                                                     "TEAM_NAME": "max",
                                                                           "SHOT_MADE_FLAG":"sum",
                                                                            "SHOT_ATTEMPTED_FLAG": "sum",
                                                                           "creates_OREB": "sum",
                                                                                    "two_next_shot_value":"sum"})

team_by_zone.rename(columns={"SHOT_ATTEMPTED_FLAG": "FGA",
                               "creates_OREB": "OREB",
                        "SHOT_MADE_FLAG":"FGM",
                            "two_next_shot_value":"2nd_Chance_Pts"}, inplace=True)
team_by_zone.fillna(0, inplace=True)
team_by_zone["FG%"] = team_by_zone.FGM/team_by_zone.FGA
team_by_zone["MISSES"] = team_by_zone.FGA - team_by_zone.FGM
team_by_zone["OREB%"] = team_by_zone.OREB/team_by_zone.MISSES
team_by_zone["shot_value"] = np.where(team_by_zone.SHOT_TYPE.str.contains('2PT') ,2,3)
team_by_zone["pts_per_OREB"] = team_by_zone["2nd_Chance_Pts"]/team_by_zone["OREB"]
team_by_zone.fillna(0, inplace=True)
team_by_zone['xpps'] = team_by_zone["FG%"] * team_by_zone.shot_value + team_by_zone["OREB%"]*team_by_zone.pts_per_OREB

team_by_zone

,TEAM_ID,combined_zone,SHOT_TYPE,TEAM_NAME,FGM,FGA,OREB,2nd_Chance_Pts,FG%,MISSES,OREB%,shot_value,pts_per_OREB,xpps
0,1610612737,Above the Break 3_Center(C)_24+ ft._3PT Field ...,3PT Field Goal,Atlanta Hawks,32,113,12,10,0.283186,81,0.148148,3,0.833333,0.973014
1,1610612737,Above the Break 3_Left Side Center(LC)_24+ ft....,3PT Field Goal,Atlanta Hawks,45,106,12,5,0.424528,61,0.196721,3,0.416667,1.355552
2,1610612737,Above the Break 3_Right Side Center(RC)_24+ ft...,3PT Field Goal,Atlanta Hawks,38,129,14,16,0.294574,91,0.153846,3,1.142857,1.059545
3,1610612737,In The Paint (Non-RA)_Center(C)_8-16 ft._2PT F...,2PT Field Goal,Atlanta Hawks,24,60,6,2,0.400000,36,0.166667,2,0.333333,0.855556
4,1610612737,In The Paint (Non-RA)_Center(C)_Less Than 8 ft...,2PT Field Goal,Atlanta Hawks,29,77,14,16,0.376623,48,0.291667,2,1.142857,1.086580
5,1610612737,In The Paint (Non-RA)_Left Side(L)_8-16 ft._2P...,2PT Field Goal,Atlanta Hawks,1,7,0,0,0.142857,6,0.000000,2,0.000000,0.285714
6,1610612737,In The Paint (Non-RA)_Right Side(R)_8-16 ft._2...,2PT Field Goal,Atlanta Hawks,2,13,3,2,0.153846,11,0.272727,2,0.666667,0.489510
7,1610612737,Left Corner 3_Left Side(L)_24+ ft._3PT Field Goal,3PT Field Goal,Atlanta Hawks,29,73,7,8,0.397260,44,0.159091,3,1.142857,1.373599
8,1610612737,Mid-Range_Center(C)_16-24 ft._2PT Field Goal,2PT Field Goal,Atlanta Hawks,14,24,2,2,0.583333,10,0.200000,2,1.000000,1.366667
9,1610612737,Mid-Range_Center(C)_8-16 ft._2PT Field Goal,2PT Field Goal,Atlanta Hawks,5,14,0,0,0.357143,9,0.000000,2,0.000000,0.714286


In [177]:
player_by_zone = shotsPBP.groupby(by=["PLAYER_NAME","combined_zone"], as_index=False).agg({"SHOT_TYPE":"max",
                                                                                           "TEAM_NAME": "max",
                                                                                           "TEAM_ID": "max",
                                                                                           "PLAYER_ID": "max",
                                                                           "SHOT_MADE_FLAG":"sum",
                                                                            "SHOT_ATTEMPTED_FLAG": "sum",
                                                                           "creates_OREB": "sum",
                                                                                          "two_next_shot_value":"sum"})

player_by_zone.rename(columns={"SHOT_ATTEMPTED_FLAG": "FGA",
                               "creates_OREB": "OREB",
                        "SHOT_MADE_FLAG":"FGM",
                            "two_next_shot_value":"2nd_Chance_Pts"}, inplace=True)
player_by_zone.fillna(0, inplace=True)
player_by_zone["FG%"] = player_by_zone.FGM/player_by_zone.FGA
player_by_zone["MISSES"] = player_by_zone.FGA - player_by_zone.FGM
player_by_zone["OREB%"] = player_by_zone.OREB/player_by_zone.MISSES
player_by_zone["shot_value"] = np.where(player_by_zone.SHOT_TYPE.str.contains('2PT') ,2,3)
player_by_zone["pts_per_OREB"] = player_by_zone["2nd_Chance_Pts"]/player_by_zone["OREB"]
player_by_zone.fillna(0, inplace=True)
player_by_zone['xpps'] = player_by_zone["FG%"] * player_by_zone.shot_value + player_by_zone["OREB%"]*player_by_zone.pts_per_OREB

player_by_zone

,PLAYER_NAME,combined_zone,SHOT_TYPE,TEAM_NAME,TEAM_ID,PLAYER_ID,FGM,FGA,OREB,2nd_Chance_Pts,FG%,MISSES,OREB%,shot_value,pts_per_OREB,xpps
0,Aaron Gordon,Above the Break 3_Center(C)_24+ ft._3PT Field ...,3PT Field Goal,Orlando Magic,1610612753,203932,4,11,1,3,0.363636,7,0.142857,3,3.000000,1.519481
1,Aaron Gordon,Above the Break 3_Left Side Center(LC)_24+ ft....,3PT Field Goal,Orlando Magic,1610612753,203932,7,18,2,0,0.388889,11,0.181818,3,0.000000,1.166667
2,Aaron Gordon,Above the Break 3_Right Side Center(RC)_24+ ft...,3PT Field Goal,Orlando Magic,1610612753,203932,3,11,1,0,0.272727,8,0.125000,3,0.000000,0.818182
3,Aaron Gordon,In The Paint (Non-RA)_Center(C)_8-16 ft._2PT F...,2PT Field Goal,Orlando Magic,1610612753,203932,4,9,0,0,0.444444,5,0.000000,2,0.000000,0.888889
4,Aaron Gordon,In The Paint (Non-RA)_Center(C)_Less Than 8 ft...,2PT Field Goal,Orlando Magic,1610612753,203932,4,13,4,8,0.307692,9,0.444444,2,2.000000,1.504274
5,Aaron Gordon,In The Paint (Non-RA)_Left Side(L)_8-16 ft._2P...,2PT Field Goal,Orlando Magic,1610612753,203932,1,4,1,3,0.250000,3,0.333333,2,3.000000,1.500000
6,Aaron Gordon,In The Paint (Non-RA)_Right Side(R)_8-16 ft._2...,2PT Field Goal,Orlando Magic,1610612753,203932,0,1,0,0,0.000000,1,0.000000,2,0.000000,0.000000
7,Aaron Gordon,Left Corner 3_Left Side(L)_24+ ft._3PT Field Goal,3PT Field Goal,Orlando Magic,1610612753,203932,5,12,3,4,0.416667,7,0.428571,3,1.333333,1.821429
8,Aaron Gordon,Mid-Range_Center(C)_16-24 ft._2PT Field Goal,2PT Field Goal,Orlando Magic,1610612753,203932,0,2,0,0,0.000000,2,0.000000,2,0.000000,0.000000
9,Aaron Gordon,Mid-Range_Center(C)_8-16 ft._2PT Field Goal,2PT Field Goal,Orlando Magic,1610612753,203932,0,7,1,0,0.000000,7,0.142857,2,0.000000,0.000000


In [181]:
league_by_zone.to_csv("league-xpps.csv")

In [182]:
team_by_zone.to_csv("team-xpps.csv")

In [183]:
player_by_zone.to_csv("player-xpps.csv")